# 软体机器人仿真：从绳长数据驱动

**目标:**
1.  读取包含机器人绝对绳长 (mm) 和对应末端真实坐标 (mm) 的 Excel 文件。
2.  手动初始化 PyBullet 仿真环境。
3.  手动初始化 ODE 求解器 (`visualizer.ODE`) 并设置参数。
4.  根据输入的绳长变化量 (`dl`) 计算曲率 (`ux`, `uy`) 和 ODE 动作 (`action`)。
5.  手动创建机器人的 PyBullet 可视化模型。
6.  在循环中应用动作，更新 ODE 状态，计算机器人新形态。
7.  手动更新 PyBullet 中机器人模型的可视化。
8.  记录每一步的输入绳长、真实坐标以及仿真得到的末端坐标。
9.  将所有记录的数据保存到新的 Excel 文件中。

**核心逻辑:** `dl -> ux, uy -> action -> ODE 更新 -> 计算形态 sol -> 手动更新 PyBullet 可视化 -> 获取仿真坐标`

**依赖:**
* `numpy`, `pandas`, `math`, `os`, `sys`, `time`
* `pybullet`
* `visualizer.py` 文件 (包含 `ODE` 类)
* `openpyxl` (用于读写 Excel)

In [ ]:
# 导入所需的库
import sys
import os
import numpy as np
import pybullet as p 
import pybullet_data 
import time
import math
from pprint import pprint
import pandas as pd
notebook_dir = os.getcwd() # 获取当前 Notebook 的工作目录
project_root = os.path.abspath(os.path.join(notebook_dir, '..')) 
if project_root not in sys.path:
    sys.path.append(project_root)
from visualizer.visualizer import ODE

In [3]:
# 数据处理函数定义

def load_and_preprocess_data(file_path, sheet_name):
    """加载 Excel 文件，提取所需数据，并进行预处理 。"""
    print(f"[信息] 正在加载数据文件: {file_path}")
    
    df_input = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    print(f"[成功] 已加载数据，共 {len(df_input)} 行。")
    
    absolute_lengths_mm = df_input[['cblen1', 'cblen2', 'cblen3']].values
    real_xyz_mm = df_input[['X', 'Y', 'Z']].values
    print(f"[信息] 成功提取 {len(absolute_lengths_mm)} 行绳长和真实 XYZ 坐标。")
    
    L0_cables_mm = absolute_lengths_mm[0] 
    print(f"[假设] 使用第一行 L0(mm): {L0_cables_mm}")
    
    absolute_lengths_m = absolute_lengths_mm / 1000.0
    L0_cables_m = L0_cables_mm / 1000.0
    dl_sequence_m = absolute_lengths_m - L0_cables_m 
    print(f"[信息] 已计算 {len(dl_sequence_m)} 行 dl (m)。")
    
    return absolute_lengths_mm, real_xyz_mm, dl_sequence_m, L0_cables_mm

def initialize_results_storage():
    """初始化用于存储仿真结果的字典。"""
    return {
        'cblen1_mm': [], 'cblen2_mm': [], 'cblen3_mm': [],
        'X_real_mm': [], 'Y_real_mm': [], 'Z_real_mm': [],
        'sim_X_mm': [], 'sim_Y_mm': [], 'sim_Z_mm': []
    }

def append_result(results_dict, cblen_mm, real_xyz_mm, sim_xyz_m):
    """将单步的输入和仿真结果追加到结果字典中。"""
    results_dict['cblen1_mm'].append(cblen_mm[0])
    results_dict['cblen2_mm'].append(cblen_mm[1])
    results_dict['cblen3_mm'].append(cblen_mm[2])
    results_dict['X_real_mm'].append(real_xyz_mm[0])
    results_dict['Y_real_mm'].append(real_xyz_mm[1])
    results_dict['Z_real_mm'].append(real_xyz_mm[2])
    # 转换仿真坐标并应用变换
    sim_x_mm = sim_xyz_m[0] * 1000.0 if not np.isnan(sim_xyz_m[0]) else np.nan
    sim_y_mm = sim_xyz_m[1] * -1000.0 if not np.isnan(sim_xyz_m[1]) else np.nan # Y 轴反向
    sim_z_mm = sim_xyz_m[2] * 1000.0 - 480 if not np.isnan(sim_xyz_m[2]) else np.nan # Z 轴偏移
    results_dict['sim_X_mm'].append(sim_x_mm)
    results_dict['sim_Y_mm'].append(sim_y_mm)
    results_dict['sim_Z_mm'].append(sim_z_mm)

def save_results_to_excel(results_dict, output_path):
    """将结果字典保存到 Excel 文件 (无错误检查简化版)。"""
    print("\n--- 保存仿真结果 ---")
    output_column_order = [
        'cblen1_mm', 'cblen2_mm', 'cblen3_mm',
        'X_real_mm', 'Y_real_mm', 'Z_real_mm',
        'sim_X_mm', 'sim_Y_mm', 'sim_Z_mm'
    ]
    output_results_df = pd.DataFrame(results_dict)
    output_results_df = output_results_df.reindex(columns=output_column_order)
    print(f"[信息] 正在将 {len(output_results_df)} 条结果 ({len(output_column_order)} 列) 保存到: {output_path}")
    
    output_results_df.to_excel(output_path, index=False, engine='openpyxl')
    print(f"[成功] 结果已保存至: {os.path.abspath(output_path)}")

print("数据处理函数定义完毕。")

数据处理函数定义完毕。


In [4]:
# 计算函数定义

def calculate_orientation(point1, point2):
    """根据两点计算方向 (四元数)"""
    diff = np.array(point2) - np.array(point1)
    norm_diff = np.linalg.norm(diff)
    if norm_diff < 1e-6:
        return p.getQuaternionFromEuler([0, 0, 0]), [0, 0, 0]
    if np.linalg.norm(diff[:2]) < 1e-6:
        yaw = 0
    else:
        yaw = math.atan2(diff[1], diff[0])
    pitch = math.atan2(-diff[2], math.sqrt(diff[0]**2 + diff[1]**2))
    roll = 0
    return p.getQuaternionFromEuler([roll, pitch, yaw]), [roll, pitch, yaw]

def calculate_curvatures_from_dl(dl_segment, d, L0_seg, num_cables=3):
    """根据绳长变化量计算曲率 ux, uy (假设输入有效且为3绳索)。"""
    ux = 0.0
    uy = 0.0
    if abs(d) < 1e-9:
        print("警告: 绳索半径 d 接近于零。")
        return ux, uy
    dl1 = dl_segment[0]
    dl2 = dl_segment[1]
    dl3 = dl_segment[2]
    uy = -dl1 / d
    denominator_ux = d * math.sqrt(3.0)
    if abs(denominator_ux) > 1e-9:
        ux = (dl3 - dl2) / denominator_ux
    else:
        ux = 0.0
        print("警告: 计算 ux 时分母接近零。")
    return ux, uy

def convert_curvatures_to_ode_action(ux, uy, length_change, d, L0_seg):
    """将曲率转换为 ODE action (3元素数组)。"""
    l = L0_seg
    action_ode = np.zeros(3)
    action_ode[0] = length_change
    action_ode[1] = uy * l * d
    action_ode[2] = -ux * l * d
    return action_ode

print("物理/计算函数定义完毕。")

物理/计算函数定义完毕。


In [ ]:
# 参数配置
print("--- 设置参数 ---")

# --- 文件路径 ---
# !!! 修改实际的文件路径 !!!
DATA_FILE_PATH = 'c:/Users/11647/Desktop/data/circle2_without_0.xlsx'
SHEET_NAME = 'Sheet1'

output_dir = os.getcwd() 
OUTPUT_RESULTS_PATH = os.path.join(output_dir, 'manual_sim_results.xlsx') # 修改输出文件名

# --- 机器人物理参数 ---
num_cables = 3
cable_distance = 0.004 
initial_length = 0.12  
number_of_segment = 1  
L0_seg = initial_length / number_of_segment 
axial_strain_coefficient = -20 # ODE 耦合系数 k_strain
AXIAL_ACTION_SCALE = 0.02      # 轴向动作缩放

# --- 可视化参数 ---
body_color = [1, 0.0, 0.0, 1]
head_color = [0.0, 0.0, 0.75, 1]
body_sphere_radius = 0.02 
number_of_sphere = 30     
my_sphere_radius = body_sphere_radius
my_number_of_sphere = number_of_sphere
my_head_color = head_color

# --- 仿真参数 ---
simulationStepTime = 0.001 
gui_enabled = True         # !!! 控制是否连接到 GUI !!!

# --- 基座参数 ---
base_pos = np.array([0, 0, 0.6])          
base_ori_euler = np.array([-math.pi / 2.0, 0, 0]) 
base_ori = p.getQuaternionFromEuler(base_ori_euler) 

print(f"输入数据文件: {DATA_FILE_PATH}")
print(f"输出结果文件: {OUTPUT_RESULTS_PATH}")
print(f"机器人参数: L0={initial_length:.4f}m, d={cable_distance:.4f}m")
print(f"ODE耦合系数 k_strain={axial_strain_coefficient}")
print(f"GUI 显示: {gui_enabled}")

In [ ]:
# 主执行逻辑

# --- 1. 加载数据 ---
print("--- 1. 加载数据 ---")
absolute_lengths_mm, real_xyz_mm, dl_sequence_m, L0_cables_mm = load_and_preprocess_data(DATA_FILE_PATH, SHEET_NAME)


# --- 2. 初始化结果存储 ---
print("\n--- 2. 初始化结果存储 ---")
results_data = initialize_results_storage()


# --- 3. PyBullet 初始化 ---
print("\n--- 3. PyBullet 初始化 ---")
physicsClientId = p.connect(p.GUI if gui_enabled else p.DIRECT) 
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)
p.setTimeStep(simulationStepTime)
planeId = p.loadURDF("plane.urdf")
print(f"加载 plane.urdf, ID: {planeId}")
if gui_enabled:
    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0) # 关闭默认 GUI 控件
    p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=45, cameraPitch=-20, cameraTargetPosition=[0, 0, 0.3])
    print("[信息] 已设置相机视角。")
else:
    print("[信息] GUI 未启用。")


# --- 4. 初始化 ODE 对象 ---
print("\n--- 4. 初始化 ODE 对象 ---")
my_ode = ODE(initial_length_m=initial_length, 
             cable_distance_m=cable_distance,
             axial_coupling_coefficient=axial_strain_coefficient)
print(f"ODE 已初始化: l0={my_ode.l0:.4f}m, d={my_ode.d:.4f}m, k_strain={my_ode.k_strain}")


# --- 5. 计算初始形态并创建机器人模型 ---
print("\n--- 5. 计算初始形态并创建机器人模型 ---")


# 5.1 计算初始形状 (dl=0)
print("计算初始形状 (dl=0)...")
act0_segment = np.zeros(3)
my_ode._reset_y0() # 重置 ODE 状态
my_ode.updateAction(act0_segment) # 应用零动作
sol0 = my_ode.odeStepFull() # 求解初始形态
if sol0 is None or sol0.shape[1] < 3:
     print("[错误] 无法计算初始形状，请检查 ODE 设置。")
     raise ValueError("无法计算初始形状")
print("初始形状计算完成。")


# 5.2 创建 PyBullet 形状
print("创建 PyBullet 形状...")
radius = my_sphere_radius
shape = p.createCollisionShape(p.GEOM_SPHERE, radius=radius)
visualShapeId = p.createVisualShape(p.GEOM_SPHERE, radius=radius, rgbaColor=body_color)
visualShapeId_tip_body = p.createVisualShape(p.GEOM_SPHERE, radius=radius+0.0025, rgbaColor=my_head_color)
visualShapeId_tip_gripper = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.002, 0.01, 0.002], rgbaColor=head_color)
print("形状创建完毕。")


# 5.3 创建 PyBullet 物体
print("创建 PyBullet 物体...")
idx0 = np.linspace(0, sol0.shape[1] - 1, my_number_of_sphere, dtype=int)
positions0_local = [(sol0[0, i], sol0[2, i], sol0[1, i]) for i in idx0] # YZ 交换适应 PyBullet
my_robot_bodies = [] # 存储所有 body ID
# 创建身体部分
for i, pos_local in enumerate(positions0_local):
    pos_world, ori_world = p.multiplyTransforms(base_pos, base_ori, pos_local, [0,0,0,1])
    bodyId = p.createMultiBody(baseMass=0.0, baseCollisionShapeIndex=-1, baseVisualShapeIndex=visualShapeId, basePosition=pos_world, baseOrientation=ori_world)
    my_robot_bodies.append(bodyId)
# 创建末端部分
ori_tip_local, _ = calculate_orientation(positions0_local[-3], positions0_local[-1]) # 依赖 positions0_local 长度
pos_tip_world, ori_tip_world = p.multiplyTransforms(base_pos, base_ori, positions0_local[-1], ori_tip_local)
bodyId_tip = p.createMultiBody(baseMass=0.0, baseCollisionShapeIndex=-1, baseVisualShapeIndex=visualShapeId_tip_body, basePosition=pos_tip_world, baseOrientation=ori_tip_world)
my_robot_bodies.append(bodyId_tip)
gripper_offset1 = [0, 0.01, 0]
pos1, _ = p.multiplyTransforms(pos_tip_world, ori_tip_world, gripper_offset1, [0,0,0,1])
bodyId_grip1 = p.createMultiBody(baseMass=0.0, baseCollisionShapeIndex=-1, baseVisualShapeIndex=visualShapeId_tip_gripper, basePosition=pos1, baseOrientation=ori_tip_world)
my_robot_bodies.append(bodyId_grip1)
gripper_offset2 = [0,-0.01, 0]
pos2, _ = p.multiplyTransforms(pos_tip_world, ori_tip_world, gripper_offset2, [0,0,0,1])
bodyId_grip2 = p.createMultiBody(baseMass=0, baseCollisionShapeIndex=-1, baseVisualShapeIndex=visualShapeId_tip_gripper, basePosition=pos2, baseOrientation=ori_tip_world)
my_robot_bodies.append(bodyId_grip2)
print(f"总共创建了 {len(my_robot_bodies)} 个物体。")


# --- 6. 运行仿真循环 ---
print("\n--- 6. 开始主仿真循环 ---")
num_data_rows = len(dl_sequence_m)
print(f"将按顺序应用 {num_data_rows} 组绳长变化量数据...")

for current_row_index in range(num_data_rows):
    # 获取当前步数据
    dl_segment = dl_sequence_m[current_row_index]
    current_cblen_mm = absolute_lengths_mm[current_row_index]
    current_real_xyz_mm = real_xyz_mm[current_row_index]

    # 计算 ODE Action
    avg_dl = np.mean(dl_segment)
    commanded_length_change = avg_dl * AXIAL_ACTION_SCALE
    ux, uy = calculate_curvatures_from_dl(dl_segment, cable_distance, L0_seg, num_cables)
    action_ode_segment = convert_curvatures_to_ode_action(ux, uy, commanded_length_change, cable_distance, L0_seg)

    # 更新 ODE 状态并计算新形状
    my_ode._reset_y0() # 每次都从初始状态开始积分（这可能与分段累积不同，请确认逻辑）
    my_ode.updateAction(action_ode_segment)
    sol = my_ode.odeStepFull() # 获取完整形状解

    # 更新 PyBullet 可视化
    pos_tip_world_m = np.array([np.nan, np.nan, np.nan]) # 初始化末端位置
    if sol is not None and sol.shape[1] >= 3:
        idx = np.linspace(0, sol.shape[1] -1, my_number_of_sphere, dtype=int)
        positions_local = [(sol[0, i], sol[2, i], sol[1, i]) for i in idx] # YZ 交换
        num_bodies_total = len(my_robot_bodies)
        num_tip_bodies = 3 # 末端球体 + 两个夹爪
        num_body_spheres = num_bodies_total - num_tip_bodies
        num_points_available = len(positions_local)
        num_spheres_to_update = min(num_body_spheres, num_points_available)

        # 更新身体部分
        for i in range(num_spheres_to_update):
            pos_world, ori_world = p.multiplyTransforms(base_pos, base_ori, positions_local[i], [0,0,0,1])
            p.resetBasePositionAndOrientation(my_robot_bodies[i], pos_world, ori_world)

        # 更新末端部分 (依赖 positions_local 长度)
        ori_tip_local, _ = calculate_orientation(positions_local[-3], positions_local[-1])
        pos_tip_world_tuple, ori_tip_world = p.multiplyTransforms(base_pos, base_ori, positions_local[-1], ori_tip_local)
        pos_tip_world_m = np.array(pos_tip_world_tuple) # 获取末端位置 (米)

        p.resetBasePositionAndOrientation(my_robot_bodies[-3], pos_tip_world_m, ori_tip_world) # 末端球体
        gripper_offset1 = [0, 0.01, 0]
        pos1, _ = p.multiplyTransforms(pos_tip_world_m, ori_tip_world, gripper_offset1, [0,0,0,1])
        p.resetBasePositionAndOrientation(my_robot_bodies[-2], pos1, ori_tip_world) # 夹爪1
        gripper_offset2 = [0,-0.01, 0]
        pos2, _ = p.multiplyTransforms(pos_tip_world_m, ori_tip_world, gripper_offset2, [0,0,0,1])
        p.resetBasePositionAndOrientation(my_robot_bodies[-1], pos2, ori_tip_world) # 夹爪2

    # 存储结果
    append_result(results_data, current_cblen_mm, current_real_xyz_mm, pos_tip_world_m)

    # 仿真步进和延时
    p.stepSimulation()
    if gui_enabled: # 只在 GUI 模式下延时以观察
        time.sleep(simulationStepTime)

    # 打印进度
    if (current_row_index + 1) % 100 == 0 or current_row_index == num_data_rows - 1:
         print(f"已处理 {current_row_index + 1}/{num_data_rows}...")


print("所有数据已处理完毕。")

In [ ]:
# 清理
print("[信息] 断开 PyBullet 连接。")
# 检查 physicsClientId 是否仍然连接，以避免断开未连接的客户端
if 'physicsClientId' in locals() and p.isConnected(physicsClientId):
     try:
         p.disconnect(physicsClientId)
         print("PyBullet 连接已断开。")
     except p.error as e:
         print(f"断开 PyBullet 时出错(可能已断开): {e}")
else:
    print("PyBullet 连接不存在或已断开。")

In [ ]:
# 保存结果
save_results_to_excel(results_data, OUTPUT_RESULTS_PATH)
print("--- 结果保存完毕 ---")

## 结束

仿真流程已完成，结果已保存到指定的 Excel 文件中。
